In [8]:
!cd py && protoc --python_out=. --proto_path=../proto/ message.proto

In [33]:
import requests
import time
host="192.168.50.116"
host="192.168.158.113"
duration_ms=10000
r = requests.get(f"http://{host}/start_snoop?duration_ms={duration_ms}", {})
print(r.text)
time.sleep(duration_ms/1000+0.1)
r = requests.get(f"http://{host}/get_snoop", {})

Snoop started. will run for 1000 buffer available: 51200


In [34]:
print(len(r.content))

0


In [2]:
import message_pb2

def parse_snoop(data):
	position = 0
	length_field = 0
	while position < len(data):
		length_field = data[position]
		position += 1
		if position + length_field > len(data):
			print("bah!")
			return
		encoded = data[position:position+length_field]
		position += length_field
		s = message_pb2.SnoopData()
		s.ParseFromString(encoded)
		yield s


In [23]:
#f = open("snoop.bin", "rb")
#cont = f.read()
f = open("snoop2.bin", "wb")
f.write(r.content)
f.close()
d=[]
#for s in parse_snoop(cont):
for s in parse_snoop(r.content):
	d.append({"prop": s.message.prop, "value":s.message.value, "src": s.metadata.source, "ts":s.metadata.recv_ms})

In [26]:
import pandas as pd
df = pd.DataFrame(d)
df

""


In [25]:
df[df.src==2].prop.unique()

AttributeError: 'DataFrame' object has no attribute 'src'